In [1]:
import numpy as np
from hyperopt import hp, fmin, tpe, space_eval
from scikits.odes.ode import ode
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [4]:
def michaelismenten(x, vm, km):
    return (vm*x)/(km+x)

def promoteractivation(x, r, kd):
    return r/(1+(x/kd))

In [ ]:
def fa_openloop(t, y, ydot, params):
    FFA, tesA = y
    r_lac, kc, km, acylacp, mu = params
    ydot[0] = tesA*((kc*acylacp))/(km+acylacp) - mu*FFA
    ydot[1] = r_lac - mu*tesA

In [ ]:
def fa_openloopintermediate(t, y, ydot, params):
    FFA, tesA, CAR = y
    r_lac, kc, km, r_bad, kc_car, km_car, acylap, mu = params
    ydot[0] = tesA * michaelismenten(acylap, kc, km) - mu * FFA - CAR * michaelismenten(FFA, kc_car, km_car)
    ydot[1] = r_lac - mu*tesA
    ydot[2] = r_bad - mu*CAR

In [ ]:
def fa_negativegeneloop(t, y, ydot, params):
    FFA, tesA, tetR = y
    r_tl, k_tesA, kd_tetR, r_tl_tetR, mu = params
    ydot[0] = tesA * k_tesA - mu * FFA
    ydot[1] = promoteractivation(tetR, r_tl, kd_tetR) - mu * tesA
    ydot[2] = promoteractivation(tetR, r_tl_tetR, kd_tetR) - mu * tetR

In [ ]:
def fa_negativemetabolicloop(t, y, ydot, params):
    FFA, tesA, FadR_t = y
    r_fl, kc, km, kd_fadR, kd_fadR_FFA, r_FadR, acylap, mu = params
    r_fl_prime = (r_fl * FadR_t)/(FadR_t + kd_fadR)
    ki = ((kd_fadR + FadR_t)/(kd_fadR))*kd_fadR_FFA
    ydot[0] = michaelismenten(acylap, kc, km) * tesA - mu*FFA
    ydot[1] = promoteractivation(FFA, r_fl_prime, ki)
    ydot[2] = r_FadR - mu * FadR_t #Not sure if need to model total FadR_t as a transient or if it can be constant

In [5]:
r_lac = hp.uniform('r_lac', 10E-11, 10E-8)
kc = hp.uniform('kc_tesA', 1, 1000)
km = hp.uniform('km_tesA', 1, 1000)
acp_mw = 1063.2 * 1000 #mg/mol, from pubChem
acylap = 43.13 / acp_mw #mg/L 
mu = 1.93E-4 #From toy model - find value from paper

#Open Loop Intermediate
r_bad = hp.uniform('r_bad', 10E-11, 10E-8)
kc_car = hp.uniform('kc_car', 2.83E-4, 4.78E-4)
km_car = hp.uniform('km_car', 2.83E-4, 4.78E-4)

#Negative gene loop
r_tl = 2.09E-11
kd_tetR = 3.0E-8
r_tl_tetR = 2.18E-11
k_tesA = 77.75

r_fl_prime = 1E-11 
kd_fadR, kd_fadR_FFA, r_FadR, acylap, mu

4.0566215199398046e-05

In [ ]:
def fa_layerednegativemetabolicloop(t, y, ydot, params):
    FFA,  tesA , tetR, FadR_t = y
    kc, km, r_tl, k_d_tetR, r_ar2, kd_fadR, kd_fadR_FFA, r_FadR, k_deg, acylap, mu = params
    ydot[0] = tesA * michaelismenten(acylap, kc, km) - mu * FFA
    ydot[1] = promoteractivation(tetR, r_tl, k_d_tetR) - mu*tesA
    ydot[2] = promoteractivation(FadR_t, r_ar2, kd_fadR * (1+FFA/kd_fadR_FFA)) - mu*tetR - k_deg * tetR
    ydot[2] = r_FadR - mu * FadR_t #Not sure if need to model total FadR_t as a transient or if it can be constant

In [ ]:
protein_synthesis_rate = [10E-11, 10E-8]
mu = 0. #FIX THIS
kc = hp.uniform('kc_tesA', 1, 1000)
km = hp.uniform('km_tesA', 1, 1000)
r_tl = 2.09E-11

kc_car = hp.uniform('kc_car', 2.83E-4, 4.78E-4)
km_car = hp.uniform('km_car', 2.83E-4, 4.78E-4)

#kd_tetR, r_tl_tetR
#r_ar2, kd_fadR, kd_fadR_FFA, r_FadR, k_deg, acylap, 